In [1]:
import warnings
warnings.filterwarnings("ignore")

In [1]:
from ray.rllib.algorithms.ppo import PPO

from esc.epanet_env import N_SIMULATION_STEPS, EPANETEnv

N_TRAINING_ITERATIONS = 1000
# Configure the algorithm.
config = {
    "env_config": {},
    # Use 5 environment workers (aka "rollout workers") that parallelly
    # collect samples from their own environment clone(s).
    "num_workers": 9,
    "horizon": N_SIMULATION_STEPS,
    # Change this to "framework: torch", if you are using PyTorch.
    # Also, use "framework: tf2" for tf2.x eager execution.
    "framework": "torch",
    # Tweak the default model provided automatically by RLlib,
    # given the environment's observation- and action spaces.
    "model": {
        "fcnet_hiddens": [64, 64],
        "fcnet_activation": "relu",
    },
    # Set up a separate evaluation worker set for the
    # `algo.evaluate()` call after training (see below).
    "evaluation_num_workers": 1,
    # Only for evaluation runs, render the env.
    # "evaluation_config": {
    #     "render_env": True,
    # },
    "log_level": "ERROR"
}

# Create our RLlib Trainer.
algo = PPO(env=EPANETEnv, config=config)

# Run it for n training iterations. A training iteration includes
# parallel sample collection by the environment workers as well as
# loss calculation on the collected batch and a model update.
for i in range(N_TRAINING_ITERATIONS):
    print(algo.train())

    if i % 50 == 0:
        checkpoint = algo.save()
        print(f"Checkpoint saved to {checkpoint}")

# Evaluate the trained Trainer (and render each timestep to the shell's
# output).
algo.evaluate()

/home/codespace/.python/current/lib/python3.10/site-packages/gym/wrappers/monitoring/video_recorder.py:9: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils.spawn
2022-11-30 23:54:07,628	WARNING algorithm.py:2531 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.
2022-11-30 23:54:07,629	WARNING ppo.py:351 -- `train_batch_size` (4000) cannot be achieved with your other settings (num_workers=9 num_envs_per_worker=1 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 445.
2022-11-30 23:54:10,151	INFO worker.py:1528 -- Started a local Ray instance.
(pid=7940) /home/codespace/.python/current/lib/python3.10/site-packages/gym/wrap

(RolloutWorker pid=7938) EPANET version 20200 loaded (EPyT version 1.0.2).
(RolloutWorker pid=7937) EPANET version 20200 loaded (EPyT version 1.0.2).
(RolloutWorker pid=7938) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=7938) 
(RolloutWorker pid=7936) EPANET version 20200 loaded (EPyT version 1.0.2).
(RolloutWorker pid=7936) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=7936) 
(RolloutWorker pid=7937) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=7937) 
(RolloutWorker pid=7940) EPANET version 20200 loaded (EPyT version 1.0.2).
(RolloutWorker pid=7944) EPANET version 20200 loaded (EPyT version 1.0.2).
(RolloutWorker pid=7944) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=7944) 
(RolloutWorker pid=7941) EPANET version 20200 loaded (EPyT version 1.0.2).
(RolloutWorker pid=7940) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=7940) 
(RolloutWorker pid=7941) Input File BUILDING.inp loaded successfully.


(pid=8231) /home/codespace/.python/current/lib/python3.10/site-packages/gym/wrappers/monitoring/video_recorder.py:9: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
(pid=8231)   import distutils.spawn
2022-11-30 23:54:18,705	INFO trainable.py:164 -- Trainable.setup took 11.078 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-11-30 23:54:18,707	WARNING util.py:66 -- Install gputil for GPU system monitoring.


(RolloutWorker pid=8231) EPANET version 20200 loaded (EPyT version 1.0.2).
(RolloutWorker pid=8231) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=8231) 
(RolloutWorker pid=7938) EPANET version 20200 loaded (EPyT version 1.0.2).
(RolloutWorker pid=7938) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=7938) 
(RolloutWorker pid=7936) EPANET version 20200 loaded (EPyT version 1.0.2).
(RolloutWorker pid=7936) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=7936) 
(RolloutWorker pid=7937) EPANET version 20200 loaded (EPyT version 1.0.2).
(RolloutWorker pid=7937) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=7937) 
(RolloutWorker pid=7940) EPANET version 20200 loaded (EPyT version 1.0.2).
(RolloutWorker pid=7940) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=7940) 
(RolloutWorker pid=7944) EPANET version 20200 loaded (EPyT version 1.0.2).
(RolloutWorker pid=7944) Input File BUILDING.inp loaded successfully.


(RolloutWorker pid=7934) /home/codespace/.python/current/lib/python3.10/site-packages/epyt/epanet.py:11809: UserWarning: WARNING: System has negative pressures.
(RolloutWorker pid=7934)   warnings.warn(errmssg.value.decode())
(RolloutWorker pid=7941) /home/codespace/.python/current/lib/python3.10/site-packages/epyt/epanet.py:11809: UserWarning: WARNING: System has negative pressures.
(RolloutWorker pid=7941)   warnings.warn(errmssg.value.decode())
(RolloutWorker pid=7938) /home/codespace/.python/current/lib/python3.10/site-packages/epyt/epanet.py:11809: UserWarning: WARNING: System has negative pressures.
(RolloutWorker pid=7938)   warnings.warn(errmssg.value.decode())
(RolloutWorker pid=7944) /home/codespace/.python/current/lib/python3.10/site-packages/epyt/epanet.py:11809: UserWarning: WARNING: System has negative pressures.
(RolloutWorker pid=7944)   warnings.warn(errmssg.value.decode())
(RolloutWorker pid=7936) /home/codespace/.python/current/lib/python3.10/site-packages/epyt/epane

...

{'custom_metrics': {}, 'episode_media': {}, 'num_recreated_workers': 0, 'info': {'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 223.18499508455236, 'cur_kl_coeff': 0.00021141412926226627, 'cur_lr': 5.0000000000000016e-05, 'total_loss': 5.632264605813449, 'policy_loss': 0.005401408399945946, 'vf_loss': 5.626860465670145, 'vf_explained_var': -0.14601497586055467, 'kl': 0.012935267614901625, 'entropy': 0.4927100000842925, 'entropy_coeff': 0.0}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 128.0}}, 'num_env_steps_sampled': 404505, 'num_env_steps_trained': 404505, 'num_agent_steps_sampled': 404505, 'num_agent_steps_trained': 404505}, 'sampler_results': {'episode_reward_max': 1565.5369995543258, 'episode_reward_min': 710.4440148001396, 'episode_reward_mean': 1227.521274526202, 'episode_len_mean': 2880.0, 'episode_media': {}, 'episodes_this_iter': 0, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_m

(RolloutWorker pid=7936) /home/codespace/.python/current/lib/python3.10/site-packages/epyt/epanet.py:11809: UserWarning: WARNING: System has negative pressures.
(RolloutWorker pid=7936)   warnings.warn(errmssg.value.decode())
(RolloutWorker pid=7943) /home/codespace/.python/current/lib/python3.10/site-packages/epyt/epanet.py:11809: UserWarning: WARNING: System has negative pressures.
(RolloutWorker pid=7943)   warnings.warn(errmssg.value.decode())
(RolloutWorker pid=7941) /home/codespace/.python/current/lib/python3.10/site-packages/epyt/epanet.py:11809: UserWarning: WARNING: System has negative pressures.
(RolloutWorker pid=7941)   warnings.warn(errmssg.value.decode())
(RolloutWorker pid=7938) /home/codespace/.python/current/lib/python3.10/site-packages/epyt/epanet.py:11809: UserWarning: WARNING: System has negative pressures.
(RolloutWorker pid=7938)   warnings.warn(errmssg.value.decode())
(RolloutWorker pid=7944) /home/codespace/.python/current/lib/python3.10/site-packages/epyt/epane

{'custom_metrics': {}, 'episode_media': {}, 'num_recreated_workers': 0, 'info': {'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 3.4072633144035134, 'cur_kl_coeff': 0.00021141412926226627, 'cur_lr': 5.0000000000000016e-05, 'total_loss': 9.923813157440513, 'policy_loss': 0.024426183000367176, 'vf_loss': 9.899386204955398, 'vf_explained_var': 0.4749440373912934, 'kl': 0.0039038952069100573, 'entropy': 0.6097841326908399, 'entropy_coeff': 0.0}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 128.0}}, 'num_env_steps_sampled': 412515, 'num_env_steps_trained': 412515, 'num_agent_steps_sampled': 412515, 'num_agent_steps_trained': 412515}, 'sampler_results': {'episode_reward_max': 1565.5369995543258, 'episode_reward_min': 710.4440148001396, 'episode_reward_mean': 1227.521274526202, 'episode_len_mean': 2880.0, 'episode_media': {}, 'episodes_this_iter': 0, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_me

(RolloutWorker pid=7940) /home/codespace/.python/current/lib/python3.10/site-packages/epyt/epanet.py:11809: UserWarning: WARNING: System has negative pressures.
(RolloutWorker pid=7940)   warnings.warn(errmssg.value.decode())
(RolloutWorker pid=7941) /home/codespace/.python/current/lib/python3.10/site-packages/epyt/epanet.py:11809: UserWarning: WARNING: System has negative pressures.
(RolloutWorker pid=7941)   warnings.warn(errmssg.value.decode())
(RolloutWorker pid=7938) /home/codespace/.python/current/lib/python3.10/site-packages/epyt/epanet.py:11809: UserWarning: WARNING: System has negative pressures.
(RolloutWorker pid=7938)   warnings.warn(errmssg.value.decode())
(RolloutWorker pid=7944) /home/codespace/.python/current/lib/python3.10/site-packages/epyt/epanet.py:11809: UserWarning: WARNING: System has negative pressures.
(RolloutWorker pid=7944)   warnings.warn(errmssg.value.decode())
(RolloutWorker pid=7936) /home/codespace/.python/current/lib/python3.10/site-packages/epyt/epane

(RolloutWorker pid=7936) EPANET version 20200 loaded (EPyT version 1.0.2).
(RolloutWorker pid=7936) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=7936) 
(RolloutWorker pid=7944) EPANET version 20200 loaded (EPyT version 1.0.2).
(RolloutWorker pid=7944) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=7944) 
(RolloutWorker pid=7938) EPANET version 20200 loaded (EPyT version 1.0.2).
(RolloutWorker pid=7938) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=7938) 
(RolloutWorker pid=7937) EPANET version 20200 loaded (EPyT version 1.0.2).
(RolloutWorker pid=7937) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=7937) 
(RolloutWorker pid=7934) EPANET version 20200 loaded (EPyT version 1.0.2).
(RolloutWorker pid=7934) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=7934) 
(RolloutWorker pid=7942) EPANET version 20200 loaded (EPyT version 1.0.2).
(RolloutWorker pid=7942) Input File BUILDING.inp loaded successfully.
